In [1]:
#!pip install celery

In [2]:
!sudo rabbitmqctl add_user myguest myguestpwd
!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

Creating user "myguest" ...
Error: user_already_exists: myguest
Setting permissions for user "myguest" in vhost "/" ...


In [3]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


This code will **run on a server machine**. It will ask worker machines to complete some tasks and send the results back to the server.

In the same folder with this code there is a whoamI.py file 
You will need to run that code on the **worker** machines:

**"celery -A whoamI worker --loglevel=info --concurrency=3" on worker machine 1** <br>
**"celery -A whoamI worker --loglevel=info --concurrency=2" on worker machine 2**

(The server machine can also be your worker machine, you will just need to run these in a terminal as new processes)

Then those processes/machines will become workers, and will be able to run the app task, i.e. in this case the echo function, whenever the broker requests it.

The echo function will be run by these commands on the workers:

from test import echo
res = echo.delay('Python rocks!'); print(type(res)); print(res)
res.result

In [4]:
from whoamI import echo

**Check the available worker machines** <br>
In the "!celery -A test status" command we use "test" because "test.py" runs on the workers

In [5]:
!celery -A whoamI status

celery@ip-172-31-31-181: OK
celery@ip-172-31-28-61: OK

2 nodes online.


**Now let us run a series of jobs asynchronously on the workers. Observe the round-robin job allocation**

In [7]:
res =[]
for iter in range(1,30):
    res.append(echo.delay('hello ' + str(iter)))

**Let us wait for the results with ".get()", and print them out**

In [8]:
for iter in range(len(res)):
    print(res[iter].get()) #let us wait for the results and print them out

Response from worker: 172.31.31.181 process_name: ForkPoolWorker-2 process_index: 1 os_pid: 16119 message: hello 1 **
Response from worker: 172.31.28.61 process_name: ForkPoolWorker-2 process_index: 1 os_pid: 4652 message: hello 2 **
Response from worker: 172.31.31.181 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 16114 message: hello 3 **
Response from worker: 172.31.28.61 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 4650 message: hello 4 **
Response from worker: 172.31.31.181 process_name: ForkPoolWorker-2 process_index: 1 os_pid: 16119 message: hello 5 **
Response from worker: 172.31.28.61 process_name: ForkPoolWorker-3 process_index: 2 os_pid: 4653 message: hello 6 **
Response from worker: 172.31.31.181 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 16114 message: hello 7 **
Response from worker: 172.31.28.61 process_name: ForkPoolWorker-1 process_index: 0 os_pid: 4650 message: hello 8 **
Response from worker: 172.31.31.181 process_name: ForkPoolWorker

**Let us run the echo function on the server instead of the workers:**

In [9]:
res=echo('running on the server')

172.31.28.61 process_name: MainProcess process_index: _none_ os_pid: 5672 message: running on the server **
